In [1]:
import pandas as pd
from pathlib import Path

data_folder = str(Path('.') / '../../input_data')
europe_data_path = str(Path(data_folder) / 'europe_data.xls')
europe_data_en_out_path = "europe_data_en.csv"
europe_data_de_out_path = "europe_data_de.csv"

In [2]:
df = pd.read_excel(europe_data_path, header=1, usecols=['Veröffentlichungs-Nummer', 'Titel', 'Zusammenfassung', 'Veröffentlichungs-Datum'])

In [3]:
df.rename(columns={'Veröffentlichungs-Nummer': 'id', 'Titel': 'title', 'Zusammenfassung': 'abstract', 'Veröffentlichungs-Datum': 'pub_date'}, inplace=True)

In [4]:
df.dropna(subset=['abstract'], inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6032 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6032 non-null   object
 1   pub_date  6032 non-null   object
 2   title     6032 non-null   object
 3   abstract  6032 non-null   object
dtypes: object(4)
memory usage: 235.6+ KB


In [6]:
from arrangement import arrange_txt_de, arrange_txt_en

In [12]:
mask_en = df['title'].str.contains(r'\[EN\]') & df['abstract'].str.contains(r'\[EN\]')
mask_de = df['title'].str.contains(r'\[DE\]') & df['abstract'].str.contains(r'\[DE\]')
df_en = df[mask_en].copy()
df_de = df[mask_de & (~mask_en)].copy()

In [13]:
df_de.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2215 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2215 non-null   object
 1   pub_date  2215 non-null   object
 2   title     2215 non-null   object
 3   abstract  2215 non-null   object
dtypes: object(4)
memory usage: 86.5+ KB


In [14]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440 entries, 36 to 9977
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3440 non-null   object
 1   pub_date  3440 non-null   object
 2   title     3440 non-null   object
 3   abstract  3440 non-null   object
dtypes: object(4)
memory usage: 134.4+ KB


In [8]:
df[mask_de & mask_en].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 135 to 9943
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        188 non-null    object
 1   pub_date  188 non-null    object
 2   title     188 non-null    object
 3   abstract  188 non-null    object
dtypes: object(4)
memory usage: 7.3+ KB


In [9]:
df_en[['title', 'abstract']] = df_en[['title', 'abstract']].apply(arrange_txt_en, axis=1)
df_de[['title', 'abstract']] = df_de[['title', 'abstract']].apply(arrange_txt_de, axis=1)

In [10]:
df_de['lang'] = 'de'
df_en['lang'] = 'en'
df = pd.concat([df_en, df_de], ignore_index=True)

In [11]:
df.dropna(subset=['abstract'], inplace=True)

In [12]:
df['lang'].value_counts() / len(df) * 100

en    58.873866
de    41.126134
Name: lang, dtype: float64

In [13]:
df['pub_date'] = pd.to_datetime(df['pub_date'], format='%d.%m.%Y')

In [14]:
df_filtered = df.sort_values(by='pub_date', ascending=False).drop_duplicates(subset='abstract')

In [15]:
df_filtered[df_filtered.duplicated('id', keep=False)].sort_values('id').tail(10)

,id,pub_date,title,abstract,lang
180,WO002024056630A1,2024-03-21,RETURN DEVICE FOR RETURNING AT LEAST ONE LINE,The invention relates to a return device (100)...,en
3584,WO002024056630A1,2024-03-21,RÜCKSTELLVORRICHTUNG ZUM RÜCKSTELLEN ZUMINDEST...,Die Erfindung betrifft eine Rückstellvorrichtu...,de
3540,WO002024068793A1,2024-04-04,"TABLAR, LAGERREGAL UND VERFAHREN ZUR AUTOMATIS...",Die Erfindung betrifft ein Tablar (10) für ein...,de
126,WO002024068793A1,2024-04-04,"TRAY, STORAGE SHELF AND METHOD FOR AUTOMATICAL...",The invention relates to a tray (10) for a sto...,en
3500,WO002024078884A1,2024-04-18,"SENSORMODUL FÜR EIN MOBILES ROBOTERSYSTEM, MOB...",Ein Sensormodul (SM) für ein mobiles Robotersy...,de
64,WO002024078884A1,2024-04-18,"SENSOR MODULE FOR A MOBILE ROBOT SYSTEM, MOBIL...",A sensor module (SM) for a mobile robot system...,en
62,WO002024079265A1,2024-04-18,"MULTICOPTER, AND ROBOT DEVICE FOR A MULTICOPTER",The invention relates to a multicopter compris...,en
3498,WO002024079265A1,2024-04-18,"MULTICOPTER, SOWIE ROTOREINRICHTUNG FÜR EINEN ...",Ein Multicopter umfasst eine Mehrzahl von Roto...,de
35,WO002024083932A1,2024-04-25,CHANGING STATION FOR THE AUTOMATIC CHANGING OF...,The invention relates to components of a chang...,en
3479,WO002024083932A1,2024-04-25,WECHSELSTATION ZUM AUTOMATISCHEN WECHSELN VON ...,Es werden Komponenten einer Wechselstation zum...,de


In [16]:
# change pub_date to pub_year
# Convert pub_date to datetime format
# Extract the year from pub_date to create pub_year
df_filtered['pub_year'] = df_filtered['pub_date'].dt.year
df_filtered['text'] = df_filtered['title'] + '. ' + df_filtered['abstract']
df_filtered[['text']].head(1)

,text
3450,Roboter mit hohlem Handgelenkelement. Roboter ...


In [17]:
df_filtered.drop(columns=['title', 'abstract'], inplace=True)

In [18]:
df_de = df_filtered[df_filtered['lang'] == 'de']
df_en = df_filtered[df_filtered['lang'] == 'en']

In [19]:
df_de.drop(columns=['lang'], inplace=True)
df_en.drop(columns=['lang'], inplace=True)

C:\Users\paul-\AppData\Local\Temp\ipykernel_32156\911583218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_de.drop(columns=['lang'], inplace=True)
C:\Users\paul-\AppData\Local\Temp\ipykernel_32156\911583218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en.drop(columns=['lang'], inplace=True)


In [20]:
df_de.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2243 entries, 3450 to 5842
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        2243 non-null   object        
 1   pub_date  2243 non-null   datetime64[ns]
 2   pub_year  2243 non-null   int64         
 3   text      2243 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 87.6+ KB


In [21]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3215 entries, 0 to 3436
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        3215 non-null   object        
 1   pub_date  3215 non-null   datetime64[ns]
 2   pub_year  3215 non-null   int64         
 3   text      3215 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 125.6+ KB


In [22]:
df_en[['id', 'pub_year', 'text']].to_csv(europe_data_en_out_path, index=False)
df_de[['id', 'pub_year', 'text']].to_csv(europe_data_de_out_path, index=False)